安装opencompass：Kaggle上已经为我们准备好了其他常用包，只需安装opencompass用于评测即可。如果不在Kaggle上运行，则还需要安装其他必要包。

In [ ]:
!pip install "opencompass[full]" transformers
# !pip install pytorch transformers datasets "opencompass[full]"

# 指令微调

In [2]:
"""
The main program for finetuning LLMs with Huggingface Transformers Library.

ALL SECTIONS WHERE CODE POSSIBLY NEEDS TO BE FILLED IN ARE MARKED AS TODO.
"""

import argparse
from dataclasses import dataclass, field
from typing import Optional, List, Dict
import sys
import torch
from transformers import TrainingArguments, HfArgumentParser, Trainer, AutoTokenizer, AutoModelForCausalLM
import datasets

In [3]:
# Define the arguments required for the main program.
# NOTE: You can customize any arguments you need to pass in.
@dataclass
class ModelArguments:
    """Arguments for model
    """
    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The path to the LLM to fine-tune or its name on the Hugging Face Hub."
        }
    )
    torch_dtype: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "Override the default `torch.dtype` and load the model under this dtype."
            ),
            "choices": ["bfloat16", "float16", "float32"],
        },
    )
    # TODO: add your model arguments here
    pass


@dataclass
class DataArguments:
    """Arguments for data
    """
    dataset_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The path to the fine-tuning dataset or its name on the Hugging Face Hub."
        }
    )
    # TODO: add your data arguments here
    max_input_length: Optional[int] = field(
        default=512,
        metadata={"help": "Maximum input length for the tokenizer."}
    )
    max_output_length: Optional[int] = field(
        default=512,
        metadata={"help": "Maximum output length for the tokenizer."}
    )

In [4]:
# The main function
# NOTE You can customize some logs to monitor your program.
def finetune():
    # TODO Step 1: Define an arguments parser and parse the arguments
    # NOTE Three parts: model arguments, data arguments, and training arguments
    # HINT: Refer to 
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/internal/trainer_utils#transformers.HfArgumentParser
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/trainer#transformers.TrainingArguments
    parser = HfArgumentParser((ModelArguments, DataArguments, TrainingArguments))
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()

    # TODO Step 2: Load tokenizer and model
    # HINT 1: Refer to
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/tokenizer#tokenizer
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/qwen2
    # HINT 2: To save training GPU memory, you need to set the model's parameter precision to half-precision (float16 or bfloat16).
    #         You may also check other strategies to save the memory!
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/llama2#usage-tips
    #   * https://huggingface.co/docs/transformers/perf_train_gpu_one
    #   * https://www.53ai.com/news/qianyanjishu/2024052494875.html
    tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_args.model_name_or_path,
        device_map="auto",
        torch_dtype="auto",  # Load model with float16 precision to save memory
        max_position_embeddings=1024,    # 最大长度1024时，batch_size=1恰好能在一张P100中训练
    )
    print(model.device)    # 确认模型加载的位置
    print(model.config)    # 确认模型当前配置

    # TODO Step 3: Load dataset
    # HINT: https://huggingface.co/docs/datasets/v3.1.0/en/package_reference/main_classes#datasets.Dataset
    dataset = datasets.load_dataset('csv', data_files=data_args.dataset_path)

    # TODO Step 4: Define the data collator function
    # NOTE During training, for each model parameter update, we fetch a batch of data, perform a forward and backward pass,
    # and then update the model parameters. The role of the data collator is to process the data (e.g., padding the data within
    # a batch to the same length) and format the batch into the input required by the model.
    #
    # In this assignment, the purpose of the custom data_collator is to process each batch of data from the dataset loaded in
    # Step 3 into the format required by the model. This includes tasks such as tokenizing the data, converting each token into 
    # an ID sequence, applying padding, and preparing labels.
    # 
    # HINT:
    #   * Before implementation, you should:
    #      1. Clearly understand the format of each sample in the dataset loaded in Step 3.
    #      2. Understand the input format required by the model (https://huggingface.co/docs/transformers/model_doc/qwen2#transformers.Qwen2ForCausalLM).
    #         Reading its source code also helps!

    def pad_or_truncate_to_target_length(model_inputs, target_length, tokenizer):
        # 获取 padding token id
        pad_token_id = tokenizer.pad_token_id

        # 处理每个序列的 input_ids，确保长度为 target_length
        input_ids = model_inputs['input_ids']
        input_ids = [seq[:target_length] + [pad_token_id] * (target_length - len(seq)) if len(seq) < target_length else seq[:target_length] for seq in input_ids]

        # 处理每个序列的 labels，确保长度为 target_length
        labels = model_inputs['labels']
        labels = [seq[:target_length] + [-100] * (target_length - len(seq)) if len(seq) < target_length else seq[:target_length] for seq in labels]

        # 处理每个序列的 attention_mask，确保长度为 target_length
        attention_mask = model_inputs['attention_mask']
        attention_mask = [seq[:target_length] + [0] * (target_length - len(seq)) if len(seq) < target_length else seq[:target_length] for seq in attention_mask]

        # 更新 model_inputs
        model_inputs['input_ids'] = input_ids
        model_inputs['labels'] = labels
        model_inputs['attention_mask'] = attention_mask

        return model_inputs
    
    # Step 4: Define the data collator function
    def data_collator(batch: List[Dict]):
        # Extract the 'instruction', 'input', and 'output' fields from the batch
        inputs = [
            (example['instruction'] or "") + (example['input'] or "")  # Ensure 'None' is replaced by empty string
            for example in batch
        ]
        targets = [(example['output'] or "") for example in batch]
    
        # Tokenize inputs and outputs (note that this is still in list form)
        model_inputs = tokenizer(inputs, max_length=data_args.max_input_length, truncation=True, padding="do_not_pad")
        labels = tokenizer(targets, max_length=data_args.max_output_length, truncation=True, padding="do_not_pad")
    
        # Add the labels to model_inputs (still using list operations)
        model_inputs['labels'] = [
            [-100] * len(input_seq) + label_seq
            for input_seq, label_seq in zip(model_inputs['input_ids'], labels['input_ids'])
        ]
        model_inputs['input_ids'] = [
            input_seq + label_seq
            for input_seq, label_seq in zip(model_inputs['input_ids'], labels['input_ids'])
        ]
        model_inputs['attention_mask'] = [
            input_mask + label_mask
            for input_mask, label_mask in zip(model_inputs['attention_mask'], labels['attention_mask'])
        ]

        # Pad or truncate the model_inputs to the target length
        model_inputs = pad_or_truncate_to_target_length(model_inputs, data_args.max_output_length, tokenizer)

#        print("Model Input:", model_inputs)  # This will print the entire model input dictionary   
    
        # Convert everything to tensor at the end
        model_inputs['input_ids'] = torch.tensor(model_inputs['input_ids'], dtype=torch.long)
        model_inputs['labels'] = torch.tensor(model_inputs['labels'], dtype=torch.long)
        model_inputs['attention_mask'] = torch.tensor(model_inputs['attention_mask'], dtype=torch.long)

        return model_inputs

#    accelerator = Accelerator()
#    model, tokenizer, dataset['train']= accelerator.prepare(
#        model, tokenizer, dataset['train']
#    )

    # TODO Step 5: Define the Trainer
    # HINT: https://huggingface.co/docs/transformers/main_classes/trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset['train'],
        data_collator=data_collator,
        tokenizer=tokenizer,
    )

#    torch.autograd.set_detect_anomaly(True)

    # Step 6: Train!
    trainer.train()
    trainer.save_model("/kaggle/working/sft_model")

In [ ]:
# Pass your training arguments.
# NOTE [IMPORTANT!!!] DO NOT FORGET TO PASS PROPER ARGUMENTS TO SAVE YOUR CHECKPOINTS!!!
sys.argv = [
    "notebook", 
    "--model_name_or_path", "/kaggle/input/qwen2.5/transformers/0.5b/1",
    "--dataset_path", "/kaggle/input/alpaca-language-instruction-training/train.csv",
    "--output_dir", "./output",
    "--num_train_epochs", "2",
    "--per_device_train_batch_size", "2",  # 增加批量大小
    "--logging_dir", "./logs",
    "--logging_steps", "10",  # 更频繁地记录日志
    "--save_steps", "500",  # 保持保存间隔
    "--save_total_limit","1",  # 最多保留1个检查点，删除较旧的
    "--learning_rate", "5e-5",  # 设置学习率
    "--gradient_accumulation_steps", "8",  # 梯度累积，模拟大批量训练
    "--adam_epsilon", "1e-8",  # 小 epsilon 提高稳定性
    "--lr_scheduler_type", "linear",  # 使用线性学习率调度器
    "--report_to", "none",  # 禁用 W&B 集成
    "--remove_unused_columns","False",#保留未使用的列
    "--eval_strategy", "no",  # 禁用评测集
#    "--gradient_clip_val", "1.0", #启用梯度裁剪
#    "--dataloader_num_workers", "2",  # 数据加载的线程数
]
#accelerator = Accelerator()
#accelerator.wait_for_everyone()
finetune()

# 评测模型

In [6]:
PLM_MODEL_PATH = "/kaggle/input/qwen2.5/transformers/0.5b/1"
SFT_MODEL_PATH = "/kaggle/working/sft_model"
torch.cuda.empty_cache()

如果你有多个GPU，可以修改下面的--hf-num-gpus参数来加速评测。

In [ ]:
!opencompass \
    --datasets hellaswag_clean_ppl winogrande_ll ARC_e_ppl ARC_c_clean_ppl SuperGLUE_BoolQ_few_shot_ppl \
    --summarizer example \
    --hf-type base \
    --hf-path {PLM_MODEL_PATH} \
    --tokenizer-kwargs padding_side="left" truncation="left" \
    --max-seq-len 2048 \
    --batch-size 4 \
    --hf-num-gpus 1 \
    --work-dir "/kaggle/working/evals/plm" \
    --debug

In [ ]:
!opencompass \
    --datasets hellaswag_clean_ppl winogrande_ll ARC_e_ppl ARC_c_clean_ppl SuperGLUE_BoolQ_few_shot_ppl \
    --summarizer example \
    --hf-type base \
    --hf-path {SFT_MODEL_PATH} \
    --tokenizer-kwargs padding_side="right" truncation="left" \
    --max-seq-len 2048 \
    --batch-size 4 \
    --hf-num-gpus 1 \
    --work-dir "/kaggle/working/evals/sft" \
    --debug